### Preparations

In [ ]:
%pip install -q mediapipe opencv-python

### Importing Libraries

In [ ]:
import cv2
from time import perf_counter
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import os

### Configuration and Initialization

In [ ]:
model_path = os.path.join(os.getcwd(), 'pose_landmarker_full.task')

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
PoseLandmarkerResult = mp.tasks.vision.PoseLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode
mp_drawing = mp.solutions.drawing_utils

### Helper Functions

#### Draw Landmarks on Image

In [ ]:
def draw_landmarks_on_image(rgb_image, detection_result):
    pose_landmarks_list = detection_result.pose_landmarks
    annotated_image = np.copy(rgb_image)
    
    # Loop through the detected poses to visualize.
    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]
        
        # Draw the pose landmarks.
        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            pose_landmarks_proto,
            solutions.pose.POSE_CONNECTIONS,
            solutions.drawing_styles.get_default_pose_landmarks_style())
    return annotated_image


#### Print Result Callback

In [ ]:
annotated_image = None

def print_result(result: PoseLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    global annotated_image
    annotated_image = draw_landmarks_on_image(output_image.numpy_view(), result)


### Main Script

#### Pose Landmarker Initialization

In [ ]:
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result
)

#### Setup and Run Pose Estimator

In [ ]:
start_time = perf_counter()

# Setup MediaPipe instance
cap = cv2.VideoCapture(0)
with PoseLandmarker.create_from_options(options) as landmarker:
    while cap.isOpened():
        ret, frame = cap.read()

        # Convert the frame received from OpenCV to a MediaPipe’s Image object
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        # Get timestamp
        elapsed_time_ms = int((perf_counter() - start_time) * 1000)

        # Make detection
        landmarker.detect_async(mp_image, elapsed_time_ms)

        # Display the annotated image if available
        if annotated_image is not None:
            cv2.imshow('Mediapipe Feed', annotated_image)
        else:
            cv2.imshow('Mediapipe Feed', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
